In [1]:
# import all ml libraries
import torch
import pandas as pd
import ankh
from torch.optim import AdamW

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
train_df = pd.read_csv('../data/ready_data/train_pdb2272.csv')

In [4]:
train_df.label.value_counts()

1    31803
0    31803
Name: label, dtype: int64

In [5]:
train_df.head()

,identifier,sequence,label,cluster
0,A0A096MJY4,MGRKKIQITRIMDERNRQVTFTKRKFGLMKKAYELSVLCDCEIALI...,1,1984
1,A0A0D2UG83,MAKSKKIVAATSGSRSRSSRAGLAFPVGRVHRLLRKGHFADRIGSG...,1,23507
2,A0A0G2JTZ2,MSSKQATSPFACTVDGEETMTQDLTSREKEEGSDQHPASHLPLHPI...,1,11680
3,A0A0G2JZ79,MIGTDPRTILKDLLPETIPPPELDDMTLWQIVINILSEPPKRKKRK...,1,23508
4,A0A0G2L7I0,MMEDEDFLLALRLQEQFDQETPAAGWPDEDCPSSKRRRVDPSGGLD...,1,14394


In [6]:
protein_sequences = train_df.sequence.tolist()
labels = train_df.label.tolist()

In [7]:
samples = protein_sequences[5:9]
labels = labels[5:9]

In [9]:
samples = [list(seq) for seq in samples]

Нужно сортировать

In [11]:
model, tokenizer = ankh.load_large_model()

In [12]:
inputs = tokenizer(samples,
                add_special_tokens=False,
                padding=True,
                is_split_into_words=True,
                return_tensors='pt',
                )

In [14]:
inputs

{'input_ids': tensor([[19,  5, 14,  ...,  4, 10,  6],
        [19,  3,  5,  ...,  0,  0,  0],
        [19, 21, 10,  ...,  0,  0,  0],
        [19,  8, 14,  ...,  0,  0,  0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')}

In [13]:
model = model.to(device)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [21]:
@torch.inference_mode()
def calculate_embed(model, inputs):
    inputs = inputs.to(device)
    output = model(**inputs)
    embeds = output.last_hidden_state
    return embeds

In [16]:
output = calculate_embed(model, inputs)

In [19]:
output.last_hidden_state.requires_grad

False

In [13]:
embeds = output.last_hidden_state

In [14]:
binary_classification_model = ankh.ConvBertForBinaryClassification(input_dim=1536,
                                                                     nhead=4,
                                                                     hidden_dim=1536,
                                                                     num_hidden_layers=1,
                                                                     num_layers=1,
                                                                     kernel_size=7,
                                                                     dropout=0.2,
                                                                     pooling='max')

In [15]:
binary_classification_model = binary_classification_model.to(device)

In [16]:
optimizer = AdamW(binary_classification_model.parameters(), lr=2e-5)

In [21]:
labels

[1, 1, 1, 1]

In [18]:
labels_tensor = torch.tensor(labels, dtype=torch.float).to(device)
labels_tensor = labels_tensor.unsqueeze(1)

In [19]:
output = binary_classification_model(embeds, labels_tensor)

In [20]:
output.loss.backward()

In [ ]:
def train(model, binary_classification_model, train_dataloader, optimizer, device):
    binary_classification_model.train()
    loss = 0.0
    for seqs, labels in train_dataloader:
        samples = [list(seq) for seq in seqs]
        inputs = tokenizer(samples,
                            add_special_tokens=False,
                            padding=True,
                            is_split_into_words=True,
                            return_tensors="pt",
                            )

        embeds = calculate_embed(model, inputs)
        labels = labels.to(device)

        optimizer.zero_grad()
        output = binary_classification_model(embeds, labels)
        output.loss.backward()
        optimizer.step()

        loss += output.loss.item()

    epoch_loss = loss / len(train_dataloader)
    return epoch_loss


In [33]:
num_epochs = 3

/tmp/ipykernel_2590931/3097516245.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.float).to(device)


tensor([[1.],
        [1.],
        [1.],
        [1.]], device='cuda:0')

In [72]:
losses = []
for _ in range(1000):
    output = binary_classification_model(embeds, labels)
    losses.append(output.loss.item())
    optimizer.zero_grad()
    optimizer.step()

6.861739158630371

In [70]:
output.loss

tensor(6.9146, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [68]:
output.loss

tensor(6.4916, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)